In [ ]:
from scipy.ndimage import rotate
from scipy.misc import face
from matplotlib import pyplot as plt

img = face()
rot = rotate(img, 30, reshape=False)

fig, ax = plt.subplots(1, 2)
ax[0].imshow(img)
ax[1].imshow(rot)

In [ ]:
import numpy as np
from scipy.ndimage import zoom


def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape * 2
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    if zoom_factor < 1:

        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    elif zoom_factor > 1:

        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    else:
        out = img
    return out

zm1 = clipped_zoom(img, 0.5)
zm2 = clipped_zoom(img, 1.5)

fig, ax = plt.subplots(1, 3)
ax[0].imshow(img)
ax[1].imshow(zm1)
ax[2].imshow(zm2)

In [ ]:
import skimage
from skimage import io
import numpy as np

print('Instagram Filter Remake: Gotham')
original_image = skimage.io.imread('D:\\MULANGA\\myproject\\photos\\phot4.jpg')
original_image = skimage.util.img_as_float(original_image)


def split_image_into_channels(image):
    red_channel = image[:, :, 0]
    green_channel = image[:, :, 1]
    blue_channel = image[:, :, 2]
    return red_channel, green_channel, blue_channel


def merge_channels(red, green, blue):
    return np.stack([red, green, blue], axis=2)

r, g, b = split_image_into_channels(original_image)
im = merge_channels(r, g, b)


def sharpen(image, a, b):
    blurred = skimage.filters.gaussian_filter(image, sigma=10, multichannel=True)
    sharper = np.clip(image * a - blurred * b, 0, 1.0)
    return sharper


def channel_adjust(channel, values):
    orig_size = channel.shape
    flat_channel = channel.flatten()
    adjusted = np.interp(flat_channel, np.linspace(0, 1, len(values)), values)
    return adjusted.reshape(orig_size)
r, g, b = split_image_into_channels(original_image)
r_interp = channel_adjust(r, [0, 0.8, 1.0])
red_channel_adj = merge_channels(r_interp, g, b)
skimage.io.imsave('D:\\MULANGA\\myproject\\photos\\adjusted.jpg', red_channel_adj)
r, g, b = split_image_into_channels(original_image)
r_boost_lower = channel_adjust(r, [0, 0.05, 0.1, 0.2, 0.3, 0.5, 0.7, 0.8, 0.9, 0.95, 1.0])
r_boost_img = merge_channels(r_boost_lower, g, b)
skimage.io.imsave('D:\\MULANGA\\myproject\\photos\\boosted.jpg', r_boost_img)

bluer_blacks = merge_channels(r_boost_lower, g, np.clip(b + 0.03, 0, 1.0))
skimage.io.imsave('D:\\MULANGA\\myproject\\photos\\blacks.jpg', bluer_blacks)

sharper = sharpen(bluer_blacks, 1.3, 0.3)
skimage.io.imsave('D:\\MULANGA\\myproject\\photos\\sharpened.jpg', sharper)
r, g, b = split_image_into_channels(sharper)
b_adjusted = channel_adjust(b, [0, 0.047, 0.118, 0.251, 0.318, 0.392, 0.42, 0.439, 0.475, 0.561, 0.58, 0.627, 0.671, 0.733, 0.847, 0.925, 1])
gotham = merge_channels(r, g, b_adjusted)
skimage.io.imsave('D:\\MULANGA\\myproject\\photos\\final.jpg', gotham)

In [ ]:
import os
from skimage.io import imread_collection
from skimage.util import montage
import matplotlib.pyplot as plt
image_dir = 'D:\\MULANGA\\myproject\\images2.jpeg'
image_collection = imread_collection(os.path.join(image_dir, '*.jpeg'))
plt.imshow(montage_image)
plt.axis('off')
plt.show()

